In [ ]:
# Convert data to python arrays

import numpy as np
import os
from enum import Enum

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

NO_MOVE = -1000

boards = []
y = []

def read_line(string):
    values = string.split(",")
    moves_string = values[0]
    evaluations = [int(values[i]) for i in range(1, 8)]
    indexes = [0 for _ in range(7)]
    matrix = np.full((7, 6), BoardFields.EMPTY.value, dtype="int8")
    turn = BoardFields.PLAYER1

    for move_string in moves_string:
        move = int(move_string) - 1
        matrix[move, indexes[move]] = turn.value
        indexes[move] += 1
        turn = (
            BoardFields.PLAYER1 if turn == BoardFields.PLAYER2 else BoardFields.PLAYER2
        )

    for i in range(len(evaluations)):
        if evaluations[i] == NO_MOVE:
            continue
        child_matrix = np.copy(matrix)
        child_matrix[i, indexes[i]] = turn.value
        boards.append(child_matrix)
        y.append(evaluations[i])


for filename in os.listdir(os.getcwd() + "/data"):
    path = os.getcwd() + "/data/" + filename
    try:
        with open(path, "r") as file:
            for line in file:
                read_line(line.strip())
    except FileNotFoundError:
        print("File not found.")
    except IOError:
        print("Error reading the file.")

np.save(os.getcwd() + "/boards.npy", boards)
np.save(os.getcwd() + "/y.npy", y)


In [ ]:
# Convert python arrays into model input

import numpy as np
import os
from enum import Enum

from convert_board import convert_board

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

boards = np.load(os.getcwd() + "/boards.npy")
positions = []
my_lines = []
enemy_lines = []

length = len(boards)
increment = 5
count = 0
percentage = 0
for board in boards:
    converted_board = convert_board(board)

    positions.append(converted_board[0])
    my_lines.append(converted_board[1])
    enemy_lines.append(converted_board[2])

    count += 1
    if count > percentage / 100 * length:
        percentage += increment
        print(percentage, "%")


np.save(os.getcwd() + "/positions.npy", positions)
np.save(os.getcwd() + "/my_lines.npy", my_lines)
np.save(os.getcwd() + "/enemy_lines.npy", enemy_lines)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Input
from tensorflow.keras.models import Model
import datetime as dt


input_shape1 = (2, 7, 6)
input_shape2 = (3,)
input_shape3 = (3,)

input1 = Input(shape=input_shape1, name="regression with lines")
input2 = Input(shape=input_shape2)
input3 = Input(shape=input_shape3)

flatten1 = Flatten()(input1)

concatenated = Concatenate()([flatten1, input2, input3])

dense1 = Dense(units=12, activation='relu')(concatenated)
dense2 = Dense(units=6, activation='relu')(dense1)
dense3 = Dense(units=2, activation='relu')(dense2)
output = Dense(units=1, activation='sigmoid')(dense3)


model = Model(inputs=[input1, input2, input3], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

model.save(f"models/{dt.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")

In [ ]:
# Train model

import numpy as np
import tensorflow as tf
from datetime import datetime
from os import listdir
from os.path import isfile, join

ys = np.load("y.npy")
positions = np.load("positions.npy")
my_lines = np.load("my_lines.npy")
enemy_lines = np.load("enemy_lines.npy")

while True:
    model_name = sorted([f for f in listdir("models") if isfile(join("models", f))])[-1]
    model = tf.keras.models.load_model(f"models/{model_name}")
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

    model.fit([positions, my_lines, enemy_lines], ys, epochs = 1, batch_size = 128, validation_split = 0.2)

    model.save(f"models/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")
